<a href="https://colab.research.google.com/github/sithin42/INT-PROSTATE-Contour-Stability/blob/main/2_RadiomicsFeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 2 - In Silico Contour Generation & Feature Extraction**

This notebook was designed to illustrate the workflow associated with the in-silico contour generation using Torchio and feature extraction using Pyradiomics

**Step 1:** Adjust the notebook for local and colab compatability

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

ROOT_PATH = "./"
#Loading the example data from github
if IN_COLAB:
  ROOT_PATH = "./INT-PROSTATE-Contour-Stability"
  !git clone https://github.com/sithin42/INT-PROSTATE-Contour-Stability.git
  import sys
  sys.path.append(ROOT_PATH)
    

  

**Step 2:** Install and import the packages

Apart from TorchIO introduced in the previous notebook, another important package used here as part the workflow is pyradiomics. This is an open-source python package for the extraction of Radiomics features from medical imaging.

More info can be found @ https://pyradiomics.readthedocs.io/en/latest/



In [ ]:
#Requirements 
!pip install torchio
!pip install SimpleITK
!pip install pyradiomics
!pip install pandas
!pip install matplotlib


In [ ]:
import os
import SimpleITK as sitk
from tqdm import tqdm
import numpy as np
import radiomics
import torchio as tio
import pandas as pd
from utils import ContourInPlaneAug, ContourOutPlaneAug, vol_dice_score, get_aug_fn

from ipywidgets import widgets, interact
import matplotlib.pyplot as plt

In [ ]:
import logging
# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)#To ignore feature warnings

**Step 3:** Loading and visualizing the data

Here PID can either be set to "PCAMPMRI-00002" or "PCAMPMRI-00001". These are the two patients extracted from QIN prostate dataset. 

If you are running this notebook locally, you can find the data inside the `data` folder.

If you are running this notebook on Colab, you can find the data inside the following location: `INT-PROSTATE-Contour-Stability/data `

In [ ]:
PID = "PCAMPMRI-00002"

img_path = os.path.join(ROOT_PATH,"data",PID,"image.nii.gz")
mask_path = os.path.join(ROOT_PATH,"data",PID,"mask.nii.gz")
  

Quick load and visualization of data using TorchIO

In [ ]:
sub = tio.Subject(img=tio.ScalarImage(img_path),mask=tio.LabelMap(mask_path))
sub.plot()#Fast Visualization

Interactive visualization

In [ ]:
#Interactive Visualization

sitk_img = sitk.ReadImage(img_path)
sitk_mask = sitk.ReadImage(mask_path)

img_arr = sitk.GetArrayFromImage(sitk_img)#Z,X,Y
mask_arr = sitk.GetArrayFromImage(sitk_mask)

spacing_W, spacing_H, _ = sitk_img.GetSpacing()

def visualize(i):
    
    plt.imshow(img_arr[i],cmap='gray')
    if mask_arr[i].sum()>0:
        plt.contour(mask_arr[i])
    plt.show()
    
interact(visualize, i=widgets.IntSlider(len(mask_arr)//2,0,len(mask_arr)-1,1))

**Step 4:** Classical Radiomic Feature Extraction Pipeline

Here we are simply extracting features using pyradiomics based on the parameter settings

If you are running this notebook locally, you can find the parameter settings inside the location: `paramSettings/`

If you are running this notebook on google colab, you can find the parameter settings inside the location: `INT-PROSTATE-Contour-Stability/paramSettings/`

In [ ]:
PATIENT_IDS = ["PCAMPMRI-00001","PCAMPMRI-00002"]
PARAM_SETTINGS = os.path.join(ROOT_PATH,"paramSettings/StudySettings3D.yaml")

In [ ]:
def extract_features(pids):

  features = []

  pbar = tqdm(range(len(PATIENT_IDS)), desc="Extracting Features")

  extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(PARAM_SETTINGS,verbosity=False)
  
  for id in pids:

    img_path = os.path.join(ROOT_PATH,"data",id,"image.nii.gz")
    mask_path = os.path.join(ROOT_PATH,"data",id,"mask.nii.gz")

    featureVector = extractor.execute(img_path,mask_path)

    featureVector["id"] = id
    featureVector["dice"] = 1.0 #because this is the ground truth ROI
    featureVector["judge"] = 0

    features.append(featureVector)

    pbar.update()

  df = pd.DataFrame(features)

  return df


Putting it all together

In [ ]:
df = extract_features(PATIENT_IDS)
if not os.path.exists("./results"):
  os.makedirs("./results")
df.to_csv("./results/org_feats.csv")
df.head()

**Step 5:** In-Silico Contour Generation & Feature Extraction

This function synthetically generate the contour dynamically based on the augmentation scenario specified and radiomic features are extracted from the image-synthetic ROI pair.

In [ ]:
def extract_aug_features(pids, aug_type, bias_type, aug_count):
    
    features = []

    pbar = tqdm(range(aug_count*len(pids)), desc="Extracting Features", position=0)

    extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(PARAM_SETTINGS,verbosity=False)
    
    for pid in pids:

      img = sitk.ReadImage(os.path.join(ROOT_PATH,"data",pid,"image.nii.gz"))
      mask = sitk.ReadImage(os.path.join(ROOT_PATH,"data",pid,"mask.nii.gz"))

      img_arr = sitk.GetArrayFromImage(img)
      mask_arr = sitk.GetArrayFromImage(mask)

      aug_fn = get_aug_fn(aug_type, bias_type, img.GetSpacing(), IN_AUG_PARAMS, OUT_AUG_PARAMS)
      subject = tio.Subject(img=tio.ScalarImage(tensor=img_arr[np.newaxis,...]),mask=tio.LabelMap(tensor=mask_arr[np.newaxis,...]))

      for i in range(aug_count):

          pbar.set_description(f"Extracting Features {pid} - Synthetic ROI #{i+1}")

          aug_subject = aug_fn(subject)
          aug_mask_arr = aug_subject["mask"]["data"][0].numpy()

          dice = vol_dice_score(aug_mask_arr,mask_arr)

          aug_mask = sitk.GetImageFromArray(aug_mask_arr)
          aug_mask.SetSpacing(img.GetSpacing())
          aug_mask.SetOrigin(img.GetOrigin())

          
          featureVector = extractor.execute(img,aug_mask)

          featureVector['id'] = pid
          featureVector['dice'] = dice
          featureVector['judge'] = i+1

          features.append(featureVector)
          pbar.update()
            
                    
    aug_df = pd.DataFrame(features)

    return aug_df


**Step 6:** Parameters associated with the augmentation scenarios and feature extraction

You can update the parameters below and re-run the cells in step (6) to test various augmentation scenarios

Please consider the following augmentation scenarios

1.   In-Plane Augmentation with Random Bias (AUG_TYPE = "in_plane", BIAS_TYPE="random")
2.   In-Plane Augmentation with systematic bias (AUG_TYPE = "in_plane", BIAS_TYPE="systematic")
3.   Out-Plane Augmentation (AUG_TYPE = "out_plane", BIAS_TYPE="")
4.   In&Out-Plane Augmentation with Random bias (AUG_TYPE = "inout_plane", BIAS_TYPE="random")
5.   In&Out-Plane augmentation with Systematic bias (AUG_TYPE = "inout_plane", BIAS_TYPE="systematic")


Let's do this!

In [ ]:
'''
IN_AUG_PARAMS: 
w_spacing and h_spacing - represents pixel spacing and will be computed dynamically; 
w_stdMM and h_stdMM - indicates the standard deviation associated with the variability of contour allowed in X and Y axis in mm
angle - specifies the bound of contour rotation allowed
ob_type - essentially is the bias type

OUT_AUG_PARAMS:
scale_a and scale_b - specifies how the shifted boundary contour needs to be scaled with respect to the closest ROI
delta_z - essentially specifies the maximum number of times the random shift can be performed on either sides. Higher number increases the likelyhood of shift operation.

AUGMENT_COUNT - specifies the number of synthetic contours that needs to be generated for a single input GT mask

AUG_TYPE - specifies the augmentation types. Can take values in_plane, out_plane, or inout_plane

BIAS TYPE - specifies the bias type. Can take values "random" or "systematic" for in or inout_plane augmentations, for out_plane augmentation the bias should be an empty string

'''

AUG_COUNT = 5
AUG_TYPE = "inout_plane"
BIAS_TYPE = "random"

IN_AUG_PARAMS = {'w_spacing':None,"h_spacing":None,'w_stdMM':2.7,'h_stdMM':2.7, 'angle':5,'bias_type':None}
OUT_AUG_PARAMS = {'scale_a':0.6,'scale_b':0.8,'angle':5,'delta_z':2}

PATIENT_IDS = ["PCAMPMRI-00001","PCAMPMRI-00002"]
PARAM_SETTINGS = os.path.join(ROOT_PATH,"paramSettings/StudySettings3D.yaml")

In [ ]:
aug_df = extract_aug_features(PATIENT_IDS,AUG_TYPE,BIAS_TYPE,AUG_COUNT)
out_path = f"./results/{AUG_TYPE}_{BIAS_TYPE}"

if not os.path.exists(out_path):
  os.makedirs(out_path)
aug_df.to_csv(os.path.join(out_path,"aug_feats.csv"))
aug_df.head()
